In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
from src.DataUpdateModule import DataUpdateModule
DUM = DataUpdateModule()

In [3]:
df_stock, df_options = DUM.read_data(datetime(2022,1,7))

In [ ]:
# Create figure with secondary y-axis for volume
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.7, 0.3],
    subplot_titles=('Stock Price', 'Volume')
)

# Add candlestick chart
fig.add_trace(
    go.Candlestick(
        x=df_stock['datetime'],
        open=df_stock['open'],
        high=df_stock['high'],
        low=df_stock['low'],
        close=df_stock['close'],
        name='Price'
    ),
    row=1, col=1
)

# Add volume bar chart
fig.add_trace(
    go.Bar(
        x=df_stock['datetime'],
        y=df_stock['volume'],
        name='Volume',
        marker_color='lightblue'
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title='Stock Price and Volume',
    xaxis_rangeslider_visible=False,
    height=600,
    showlegend=True
)

# Update axes labels
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_xaxes(title_text="Date/Time", row=2, col=1)

fig.show()

In [ ]:
# prepare dataframe sorted by time so animation frames are chronological
df_anim = df_options.sort_values("time").copy()
df_anim["time_str"] = df_anim["time"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Reshape data to have separate rows for call and put volumes
df_call = df_anim.copy()
df_call["volume"] = df_call["callVolume"]
df_call["type"] = "Call Volume"

df_put = df_anim.copy()
df_put["volume"] = df_put["putVolume"]
df_put["type"] = "Put Volume"

df_combined = pd.concat([df_call, df_put])

ymax = max(df_anim["putVolume"].max(), df_anim["callVolume"].max()) * 1.1

# Create a sorted list of unique times to ensure proper ordering
unique_times = df_anim.sort_values("time")["time_str"].unique()

fig = px.line(
    df_combined.sort_values("strike"),
    x="strike",
    y="volume",
    color="type",
    animation_frame="time_str",
    range_y=[0, ymax],
    labels={"volume": "Volume", "strike": "Strike", "time_str": "Time", "type": "Type"},
    title="Call and Put Volume across Strikes over Time",
    category_orders={"time_str": list(unique_times)}  # Enforce chronological order
)

# Add vertical lines for stock price at each time point
for i, frame in enumerate(fig.frames):
    time_str = frame.name
    stock_price = df_anim[df_anim["time_str"] == time_str]["stockPrice"].iloc[0]
    
    # Add vertical line to each frame
    frame.data = frame.data + (go.Scatter(
        x=[stock_price, stock_price],
        y=[0, ymax],
        mode="lines",
        line=dict(color="red", width=2, dash="dash"),
        name="Stock Price",
        showlegend=(i == 0)  # Only show legend on first frame
    ),)

# Add vertical line to initial plot as well
stock_price_initial = df_anim[df_anim["time_str"] == unique_times[0]]["stockPrice"].iloc[0]
fig.add_trace(go.Scatter(
    x=[stock_price_initial, stock_price_initial],
    y=[0, ymax],
    mode="lines",
    line=dict(color="red", width=2, dash="dash"),
    name="Stock Price"
))

fig.update_layout(xaxis_title="Strike", yaxis_title="Volume", transition={"duration": 300})
fig.show()

In [ ]:
# prepare dataframe sorted by time so animation frames are chronological
df_anim = df_options.sort_values("time").copy()
df_anim["time_str"] = df_anim["time"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Calculate mid prices
df_anim["callMidPrice"] = (df_anim["callAskPrice"] + df_anim["callBidPrice"]) / 2
df_anim["putMidPrice"] = (df_anim["putAskPrice"] + df_anim["putBidPrice"]) / 2

# Reshape data to have separate rows for call and put mid prices
df_call = df_anim.copy()
df_call["midPrice"] = df_call["callMidPrice"]
df_call["type"] = "Call Mid Price"

df_put = df_anim.copy()
df_put["midPrice"] = df_put["putMidPrice"]
df_put["type"] = "Put Mid Price"

df_combined = pd.concat([df_call, df_put])

ymax = max(df_anim["callMidPrice"].max(), df_anim["putMidPrice"].max()) * 1.1

# Create a sorted list of unique times to ensure proper ordering
unique_times = df_anim.sort_values("time")["time_str"].unique()

fig = px.line(
    df_combined.sort_values("strike"),
    x="strike",
    y="midPrice",
    color="type",
    animation_frame="time_str",
    range_y=[0, ymax],
    labels={"midPrice": "Mid Price", "strike": "Strike", "time_str": "Time", "type": "Type"},
    title="Call and Put Mid Prices across Strikes over Time",
    category_orders={"time_str": list(unique_times)}  # Enforce chronological order
)

# Add vertical lines for stock price at each time point
for i, frame in enumerate(fig.frames):
    time_str = frame.name
    stock_price = df_anim[df_anim["time_str"] == time_str]["stockPrice"].iloc[0]
    
    # Add vertical line to each frame
    frame.data = frame.data + (go.Scatter(
        x=[stock_price, stock_price],
        y=[0, ymax],
        mode="lines",
        line=dict(color="red", width=2, dash="dash"),
        name="Stock Price",
        showlegend=(i == 0)  # Only show legend on first frame
    ),)

# Add vertical line to initial plot as well
stock_price_initial = df_anim[df_anim["time_str"] == unique_times[0]]["stockPrice"].iloc[0]
fig.add_trace(go.Scatter(
    x=[stock_price_initial, stock_price_initial],
    y=[0, ymax],
    mode="lines",
    line=dict(color="red", width=2, dash="dash"),
    name="Stock Price"
))

fig.update_layout(xaxis_title="Strike", yaxis_title="Mid Price", transition={"duration": 300})
fig.show()

In [27]:
# prepare dataframe sorted by time so animation frames are chronological
df_anim = df_options.sort_values("time").copy()
df_anim["time_str"] = df_anim["time"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Calculate bid-ask spreads
df_anim["callSpread"] = df_anim["callAskPrice"] - df_anim["callBidPrice"]
df_anim["putSpread"] = df_anim["putAskPrice"] - df_anim["putBidPrice"]

# Reshape data to have separate rows for call and put spreads
df_call = df_anim.copy()
df_call["spread"] = df_call["callSpread"]
df_call["type"] = "Call Bid-Ask Spread"

df_put = df_anim.copy()
df_put["spread"] = df_put["putSpread"]
df_put["type"] = "Put Bid-Ask Spread"

df_combined = pd.concat([df_call, df_put])

ymax = max(df_anim["callSpread"].max(), df_anim["putSpread"].max()) * 1.1

# Create a sorted list of unique times to ensure proper ordering
unique_times = df_anim.sort_values("time")["time_str"].unique()

fig = px.line(
    df_combined.sort_values("strike"),
    x="strike",
    y="spread",
    color="type",
    animation_frame="time_str",
    range_y=[0, ymax],
    labels={"spread": "Bid-Ask Spread", "strike": "Strike", "time_str": "Time", "type": "Type"},
    title="Call and Put Bid-Ask Spreads across Strikes over Time",
    category_orders={"time_str": list(unique_times)}  # Enforce chronological order
)

# Add vertical lines for stock price at each time point
for i, frame in enumerate(fig.frames):
    time_str = frame.name
    stock_price = df_anim[df_anim["time_str"] == time_str]["stockPrice"].iloc[0]
    
    # Add vertical line to each frame
    frame.data = frame.data + (go.Scatter(
        x=[stock_price, stock_price],
        y=[0, ymax],
        mode="lines",
        line=dict(color="red", width=2, dash="dash"),
        name="Stock Price",
        showlegend=(i == 0)  # Only show legend on first frame
    ),)

# Add vertical line to initial plot as well
stock_price_initial = df_anim[df_anim["time_str"] == unique_times[0]]["stockPrice"].iloc[0]
fig.add_trace(go.Scatter(
    x=[stock_price_initial, stock_price_initial],
    y=[0, ymax],
    mode="lines",
    line=dict(color="red", width=2, dash="dash"),
    name="Stock Price"
))

fig.update_layout(xaxis_title="Strike", yaxis_title="Bid-Ask Spread", transition={"duration": 300})
fig.show()